Simple implementation of an neural network of N size:
    - where N is the number of layers that constitute the network.

Unlike previous iterations of the net we have created, for this one we would have to modularize the initialization of the network's parameters, as well as make the forward and backward passes that handles those N layers.

Also, unlike previous iterations of the net we have created, this one will include 'batch normalization', which is used to maintain a fixed distibution of the values of each layer of the network as the network trains. This mitigates 'Internal Covariate Shift', which is the phenomenon that describes the change in the distribution of the network activations due to change in network parameters.

We will also implement dropout, which drops random neurons in each layer during training, so as to help mitigate overfitting, as well as allow neurons which would not commonly be used for training to be used more readily, so as to reduce co-adaptation of the neurons.

We will ask the user for the number of total layers for the network to be created, as well as the size of each of its constituent hidden layers.

To assist in helping one to follow the code, for our user inputs for the number of layers and the size of each of the hidden layers, we will default to:

            num_layers = 5
            H_dims = size = 100, for each layer.

One can adjust, but it will be easier to follow the code with these values in mind.

In [105]:
import numpy as np
import random

# Initialization of the network parameters input X, and the labels y, as well as hidden dims.
np.random.seed(231)
num_layers = int(input('how many layers do you want in your network?'))
H_dims = [int(input('size of layer'+str(x))) for x in range(1, num_layers)]
N, D, C = 50, 3*32*32, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

# Some defualt hyperparameters to be used.
weight_scale = 5e-2
eps = 1e-5
reg_strength = 0.01
learning_rate = 0.2
momentum = 0.9
model_params = {}

N layer NeuralNet class implementation.

In [106]:
# Network architecture: 
# (affine-batchnorm-relu) * num_layers-1 -> (affine-softmax)

class NeuralNet:
    def __init__(self):
        pass

    def affine_forward(self, x, w, b):
        # Input x's second dimension is reshaped to reflect D dimensional vector.
        out = x.reshape(x.shape[0], -1).dot(w) + b
        cache = (x, w, b)
        return out, cache
    
    def affine_backward(self, dout, cache):
        # Basic derivative/chain rule rules being applied here.
        x, w, b = cache
        dx = dout.dot(w.T)
        dw = x.T.dot(dout)
        db = np.sum(dout, axis=0)
        return dx, dw, db
    
    def relu_forward(self, x):
        # If value in x is > 0, return x, otherwise 0.
        out = np.maximum(0, x)
        cache = x
        return out, cache
    
    def relu_backward(self, dout, cache):
        # Since derivatiive of relu is 1 if x > 0, just return dout is x > 0.
        x = cache
        return dout * (x > 0)
    
    def batchnorm_forward(self, x, gamma, beta, eps):
        # Do the normalization of input x: batch mean, variance, std.
        b_mean = np.mean(x, axis=0)
        b_var = np.var(x, axis=0)
        b_std = np.sqrt(b_var + eps)
        x_norm = (x - b_mean) / b_std

        # Applying linear transformation on the normalized x. (gamma and beta):
        out = (gamma * x_norm) + beta
        cache = (x, x_norm, gamma, beta, b_mean, b_var, b_std, eps)
        return out, cache
    
    def batchnorm_backward(self, dout, cache):
        # Computation of the gradients of x, gamma, and beta.
        x, x_norm, gamma, beta, b_mean, b_var, b_std, eps = cache
        dbeta = np.sum(dout, axis=0)                                            # derivative of beta wrt. dout. 
        dgamma = np.sum(x_norm * gamma, axis=0)                                 # derivative of gamma wrt. dout.
        dx_norm =  dout * gamma                                                 # derivative of x_norm wrt. dout (mult. rule)
        d_std = -np.sum(dx_norm * (x - b_mean), axis=0) / (b_std**2)            # derivative of std.
        d_variance = 0.5 *  d_std / b_std                                       # derivative of variance.
        dx1  = dx_norm / b_std + 2 * (x - b_mean) * d_variance / len(dout)      # derivative of dx1. 
        d_mean = -np.sum(dx1, axis=0)                                           # derivative of the mean.
        dx2 = d_mean / len(dout)                                                # derivative of dx2.
        dx = dx1 + dx2
        return dx, dgamma, dbeta
    
    def print_mean_std(self, x):
        # Take the value from the batchnorm forward pass, and calculate the mean and std.
        # Mean should be close to 0, std should be close to 1.
        # This is used to check if batchnorm_forward is implemented correctly.
        mean = np.mean(x, axis=0)
        std = np.sqrt(np.var(x, axis=0) + eps)
        return mean, std

    def softmax(self, x, y):
        # Computation of the softmax, used to compute the loss and gradients of the loss wrt. the scores.
        exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
        softmax = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        correct_scores = np.zeros(x.shape)
        correct_scores[range(len(y)), y] = 1
        loss = np.mean(-np.log(np.sum(softmax * correct_scores, axis=1, keepdims=True)))
        dL = (softmax - correct_scores) / len(softmax)
        return loss, dL

Now, lets implement a class that will comprise of the forward pass and backward passes. Then, It will optimize the gradients by applying one of the various optimizers (sgd, momentum, etc.) to update the network's parameters. This is what makes the network learn!

Following the network's architechure, we will:

    -compute the affine, batch, and relu passes. 

    -then at the end we will do affine (to compute scores) to softmax (to compute loss). 
    
    -once loss and gradients of the loss wrt. scores is computed, backward propogate through the network. 
    
    -once you reach end of backprop, update the parameters (using one of various optimizers). 
    
    -then forward prop again. Continue until loss decreases.

In [107]:
# For simplicity, lets make a new class called 'Solver', which will constitute the forward and backward passes.

class Solver:
    def __init__(self, X, y, N, D, num_layers, H_dims, learning_rate, weight_scale, reg_strength):
        # Learning rate. We can try different rates to see which performs best.
        pass

    def param_init(self):
        # Weight and Biases initialization. Can handle an arbitrary number of hidden dimensions.
        # Also, addition of gamma and beta parameters, which are used for the batch normalization .
        for layer, hidden_dims in enumerate(H_dims):
            if layer == 0:
                model_params['W'+str(layer+1)] = weight_scale * np.random.randn(D, hidden_dims)
                model_params['b'+str(layer+1)] = np.zeros(hidden_dims)
                model_params['gamma'+str(layer+1)] = np.ones(hidden_dims)
                model_params['beta'+str(layer+1)] = np.zeros(hidden_dims)
            else:
                model_params['W'+str(layer+1)] = weight_scale * np.random.randn(H_dims[layer-1], hidden_dims)
                model_params['b'+str(layer+1)] = np.zeros(hidden_dims)
                model_params['gamma'+str(layer+1)] = np.ones(hidden_dims)
                model_params['beta'+str(layer+1)] = np.zeros(hidden_dims)
        model_params['W'+str(num_layers)] = weight_scale * np.random.randn(H_dims[-1], C)
        model_params['b'+str(num_layers)] = np.zeros(C)
        return model_params

    def forward_pass(self, model_params):
        # Forward pass.
        nn = NeuralNet()
        # Lists that holds cache values of each part of the layers. (affine, batch, and relu caches). Used for backprop.
        layer_out = []
        affine_cache = []
        batch_cache = []
        relu_cache = []

        # Loop that computes the forward pass an arbitrary amount of times (number of layers)
        for layer in range(num_layers - 1):
            # If the first layer, use input X in the affine_forward() method.
            if layer == 0:
                w, b = model_params['W'+str(layer+1)], model_params['b'+str(layer+1)]
                a_values, a_cache_values = nn.affine_forward(X, w, b)
                
            # Otherwise, input x is the value of the relu activation function, which is in the layer_out list.
            else:
                x, w, b = layer_out[layer-1], model_params['W'+str(layer+1)], model_params['b'+str(layer+1)]
                a_values, a_cache_values = nn.affine_forward(x, w, b)

            # Apply batchnorm to the affine forward output.
            gamma, beta = model_params['gamma'+str(layer+1)], model_params['beta'+str(layer+1)]
            b_values, b_cache_values = nn.batchnorm_forward(a_values, gamma, beta, eps)

            # Compute the relu activation function using output from batchnorm. Append relu output in layer_out list.
            r_values, r_cache_values = nn.relu_forward(b_values)
            layer_out.append(r_values)
            
            # Append the cache values from the affine, batch, and relu passes into its respective cache lists.
            affine_cache.append(a_cache_values)
            batch_cache.append(b_cache_values)
            relu_cache.append(r_cache_values)

        # For the last layer, compute affine, append last layer cache, then compute the loss and dL
        x, w, b = layer_out[-1], model_params['W'+str(num_layers)], model_params['b'+str(num_layers)]
        a_values, a_cache_values = nn.affine_forward(x, w, b)
        affine_cache.append(a_cache_values)
        loss, dL = nn.softmax(a_values, y)
        #loss += 0.05 * reg_strength * np.sum([np.sum(W**2) for key, W in model_params.items() if 'W' in key])
        return loss, dL, affine_cache, batch_cache, relu_cache

    def backward_pass(self, loss, dL, affine_cache, batch_cache, relu_cache):
        # Now, lets implement the backward pass, with batchnorm in mind.
        # This is a improved version of the above backward pass.
        grads = {}
        doutx = dL

        # Loop backwards through the layers, computing the gradients and updating the grads dict.
        for layer in reversed(range(num_layers)):
            doutx, grads['W'+str(layer+1)], grads['b'+str(layer+1)] = nn.affine_backward(doutx, affine_cache[layer])
            #grads['W'+str(layer+1)] *= (reg_strength * model_params['W'+str(layer+1)])

        # As long as the layer is not the first one, compute the relu backward pass, then batchnorm backward pass.
            if layer != 0:
                doutx = nn.relu_backward(doutx, relu_cache[layer-1])
                doutx, grads['gamma'+str(layer)], grads['beta'+str(layer)] = nn.batchnorm_backward(doutx, batch_cache[layer-1])
        return grads
    
    def sgd(self, grads, model_params):
        # This is simple stochastic gradient descent. Update the parameters (weights, biases, gamma and beta).
        for layer in range(num_layers-1):
                model_params['W'+str(layer+1)] -= learning_rate * grads['W'+str(layer+1)]
                model_params['b'+str(layer+1)] -= learning_rate * grads['b'+str(layer+1)]
                model_params['gamma'+str(layer+1)] -= learning_rate * grads['gamma'+str(layer+1)]
                model_params['beta'+str(layer+1)] -= learning_rate * grads['beta'+str(layer+1)]
        model_params['W'+str(layer+1)] -= learning_rate * grads['W'+str(layer+1)]
        model_params['b'+str(layer+1)] -= learning_rate * grads['b'+str(layer+1)]
        return model_params
    
    def gradient_descent(self, grads, model_params, momentum):
        # This will be sgd + momentum. 
        for layer in range(num_layers):
            W = model_params['W'+str(layer+1)]
            grad_w = grads['W'+str(layer+1)]
            v = np.zeros_like(W)

            # Velocity update.
            v = momentum * v - learning_rate * grad_w
            W = W + v
            model_params['W'+str(layer+1)] = W
        return model_params

Optimize, update, repeat. Experimentation with different hyperparameters (particularly the learning rate), as well as varying optimizers (sgd vs sgd+momentum, vs. rmsprop, etc.)

Note: when using batch normalization, you can make the learning rate higher than if you werent using it.

In [108]:
# Lets update the weights using the gradients. We will use sgd first. Try different hyperparameters.
iterations = 100

learning_scale = np.geomspace(1e-5, 1e5, num=100)
weights_scale = np.linspace(1e-2, 9e-2, num=100)
loss_history = {}

for learning_rate, weight_scale in zip(learning_scale, weights_scale):
    solver = Solver(X, y, N, D, num_layers, H_dims, learning_rate, weight_scale, reg_strength)
    model_params = solver.param_init()
    for i in range(iterations):
        loss, dL, affine_cache, batch_cache, relu_cache = solver.forward_pass(model_params)
        grads = solver.backward_pass(loss, dL, affine_cache, batch_cache, relu_cache)
        model_params = solver.sgd(grads, model_params)  # note, since we are not taking batches, its technically batch grad descent.
        if i == 0 or i % 10 == 0:
            print('iterations: ', i, 'lr: ', learning_rate, 'weight_scale: ', weight_scale, 'loss: ', loss)
    loss_history[(learning_rate, weight_scale)] = loss

best_params = [params for params, loss in loss_history.items() if loss == min(loss_history.values())]
print(f'min loss found: {min(loss_history.values())}')
print(f'best parameters found: {best_params}')

iterations:  0 lr:  1e-05 weight_scale:  0.01 loss:  2.3002850671502433
iterations:  10 lr:  1e-05 weight_scale:  0.01 loss:  2.299409978433121
iterations:  20 lr:  1e-05 weight_scale:  0.01 loss:  2.298530755103659
iterations:  30 lr:  1e-05 weight_scale:  0.01 loss:  2.2976562799308384
iterations:  40 lr:  1e-05 weight_scale:  0.01 loss:  2.29678781909413
iterations:  50 lr:  1e-05 weight_scale:  0.01 loss:  2.2959219065598484
iterations:  60 lr:  1e-05 weight_scale:  0.01 loss:  2.2950542234058386
iterations:  70 lr:  1e-05 weight_scale:  0.01 loss:  2.29418849496549
iterations:  80 lr:  1e-05 weight_scale:  0.01 loss:  2.2933235340113494
iterations:  90 lr:  1e-05 weight_scale:  0.01 loss:  2.2924600349330175
iterations:  0 lr:  1.2618568830660211e-05 weight_scale:  0.010808080808080808 loss:  2.3011904749016985
iterations:  10 lr:  1.2618568830660211e-05 weight_scale:  0.010808080808080808 loss:  2.300240148860374
iterations:  20 lr:  1.2618568830660211e-05 weight_scale:  0.010808

/var/folders/5y/b0xs89p96wg8lzcfm69hdsm40000gn/T/ipykernel_2274/2789001270.py:73: RuntimeWarning: divide by zero encountered in log
  loss = np.mean(-np.log(np.sum(softmax * correct_scores, axis=1, keepdims=True)))


iterations:  30 lr:  7742.636826811277 weight_scale:  0.0811111111111111 loss:  2.3025850929940455
iterations:  40 lr:  7742.636826811277 weight_scale:  0.0811111111111111 loss:  2.3025850929940455
iterations:  50 lr:  7742.636826811277 weight_scale:  0.0811111111111111 loss:  2.3025850929940455
iterations:  60 lr:  7742.636826811277 weight_scale:  0.0811111111111111 loss:  2.3025850929940455
iterations:  70 lr:  7742.636826811277 weight_scale:  0.0811111111111111 loss:  2.3025850929940455
iterations:  80 lr:  7742.636826811277 weight_scale:  0.0811111111111111 loss:  2.3025850929940455
iterations:  90 lr:  7742.636826811277 weight_scale:  0.0811111111111111 loss:  2.3025850929940455
iterations:  0 lr:  9770.099572992247 weight_scale:  0.08191919191919192 loss:  2.3892169244488572
iterations:  10 lr:  9770.099572992247 weight_scale:  0.08191919191919192 loss:  2.3025850929940455
iterations:  20 lr:  9770.099572992247 weight_scale:  0.08191919191919192 loss:  2.3025850929940455
iteratio